<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/JFriedel_IoT_Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JFriedel                  IOT                 Assignment 6          18 Feb. 2025

ChatGPT generated code , generated 18 February 2025

In [ ]:
import os
import random
from PIL import Image, ImageDraw

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")

# Create data directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

shapes = ["circle", "square", "triangle"]

# Function to generate a random color
def random_color():
    return tuple(random.randint(0, 255) for _ in range(3))

# Function to draw a circle
def draw_circle(draw, size):
    x0 = random.randint(0, size - 50)
    y0 = random.randint(0, size - 50)
    diameter = random.randint(20, min(size - x0, size - y0))
    draw.ellipse([x0, y0, x0 + diameter, y0 + diameter], fill=random_color())

# Function to draw a square
def draw_square(draw, size):
    x0 = random.randint(0, size - 50)
    y0 = random.randint(0, size - 50)
    side = random.randint(20, min(size - x0, size - y0))
    draw.rectangle([x0, y0, x0 + side, y0 + side], fill=random_color())

# Function to draw a triangle
def draw_triangle(draw, size):
    x1 = random.randint(0, size)
    y1 = random.randint(0, size)
    x2 = random.randint(0, size)
    y2 = random.randint(0, size)
    x3 = random.randint(0, size)
    y3 = random.randint(0, size)
    draw.polygon([x1, y1, x2, y2, x3, y3], fill=random_color())

# Function to generate images
def generate_images(shape, count, size=256):
    shape_dir = os.path.join(data_dir, shape)
    if not os.path.exists(shape_dir):
        os.makedirs(shape_dir)

    for i in range(count):
        image = Image.new("RGB", (size, size), (255, 255, 255))
        draw = ImageDraw.Draw(image)

        if shape == "circle":
            draw_circle(draw, size)
        elif shape == "square":
            draw_square(draw, size)
        elif shape == "triangle":
            draw_triangle(draw, size)

        image_path = os.path.join(shape_dir, f"{shape}_{i + 1}.png")
        image.save(image_path)

# Generate 100 images for each shape
for shape in shapes:
    generate_images(shape, 100)

print("Image generation complete!")

In [ ]:
import shutil
validation_dir = os.path.join(base_dir, "validation data")
# Function to randomly select and transfer images to validation directory
# Create validation directory if it doesn't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
def transfer_validation_images(shape, count):
    shape_dir = os.path.join(data_dir, shape)
    files = [f for f in os.listdir(shape_dir) if os.path.isfile(os.path.join(shape_dir, f))]
    selected_files = random.sample(files, count)

    for file in selected_files:
        src_path = os.path.join(shape_dir, file)
        dst_path = os.path.join(validation_dir, file)
        shutil.move(src_path, dst_path)
for shape in shapes:
    transfer_validation_images(shape, 10)
print("validation transfer complete!")

In [ ]:
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

validation_dir = os.path.join(base_dir, "validation data")

# Define transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load datasets
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class ShapeClassifier(nn.Module):
    def __init__(self):
        super(ShapeClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = ShapeClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

# Evaluate the model on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate the model on validation data
class_correct = [0] * 3
class_total = [0] * 3
with torch.no_grad():
    for images, labels in validation_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(3):
    print(f"Accuracy of {shapes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%")

print("Validation complete!")

In [ ]:
import os
import random
import shutil
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")
validation_dir = os.path.join(base_dir, "validation data")

# Create validation directory and class subdirectories if they don't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
    for shape in ["circle", "square", "triangle"]:
        os.makedirs(os.path.join(validation_dir, shape), exist_ok=True)

shapes = ["circle", "square", "triangle"]

# Define transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load datasets
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class ShapeClassifier(nn.Module):
    def __init__(self):
        super(ShapeClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = ShapeClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

# Evaluate the model on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate the model on validation data
class_correct = [0] * 3
class_total = [0] * 3
with torch.no_grad():
    for images, labels in validation_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(3):
    print(f"Accuracy of {shapes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%")

print("Validation complete!")

In [ ]:
import os
import random
import shutil
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")
validation_dir = os.path.join(base_dir, "validation data")

# Create validation directory and class subdirectories if they don't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
for shape in ["circle", "square", "triangle"]:
    shape_dir = os.path.join(validation_dir, shape)
    if not os.path.exists(shape_dir):
        os.makedirs(shape_dir)

shapes = ["circle", "square", "triangle"]

# Define transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load datasets
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class ShapeClassifier(nn.Module):
    def __init__(self):
        super(ShapeClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = ShapeClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

# Evaluate the model on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate the model on validation data
class_correct = [0] * 3
class_total = [0] * 3
with torch.no_grad():
    for images, labels in validation_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(3):
    print(f"Accuracy of {shapes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%")

print("Validation complete!")

In [ ]:
import os
import random
import shutil
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")
validation_dir = os.path.join(base_dir, "validation data")

# Create validation directory and class subdirectories if they don't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
for shape in ["circle", "square", "triangle"]:
    shape_dir = os.path.join(validation_dir, shape)
    if not os.path.exists(shape_dir):
        os.makedirs(shape_dir)

# Automatically copy images to validation subdirectories
for shape in ["circle", "square", "triangle"]:
    shape_data_dir = os.path.join(data_dir, shape)
    shape_validation_dir = os.path.join(validation_dir, shape)
    if os.path.exists(shape_data_dir):
        images = [f for f in os.listdir(shape_data_dir) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        selected_images = random.sample(images, min(10, len(images)))
        for image in selected_images:
            shutil.copy(os.path.join(shape_data_dir, image), os.path.join(shape_validation_dir, image))

shapes = ["circle", "square", "triangle"]

# Define transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load datasets
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class ShapeClassifier(nn.Module):
    def __init__(self):
        super(ShapeClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = ShapeClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

# Evaluate the model on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate the model on validation data
class_correct = [0] * 3
class_total = [0] * 3
with torch.no_grad():
    for images, labels in validation_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(3):
    print(f"Accuracy of {shapes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%")

print("Validation complete!")

In [ ]:
import os
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")
validation_dir = os.path.join(base_dir, "validation data")

# Create validation directory and class subdirectories if they don't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
for shape in ["circle", "square", "triangle"]:
    shape_dir = os.path.join(validation_dir, shape)
    if not os.path.exists(shape_dir):
        os.makedirs(shape_dir)

# Automatically copy images to validation subdirectories
for shape in ["circle", "square", "triangle"]:
    shape_data_dir = os.path.join(data_dir, shape)
    shape_validation_dir = os.path.join(validation_dir, shape)
    if os.path.exists(shape_data_dir):
        images = [f for f in os.listdir(shape_data_dir) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        selected_images = random.sample(images, min(10, len(images)))
        for image in selected_images:
            shutil.copy(os.path.join(shape_data_dir, image), os.path.join(shape_validation_dir, image))

# Load datasets
batch_size = 32
img_size = (256, 256)

train_dataset = image_dataset_from_directory(data_dir, shuffle=True, batch_size=batch_size, image_size=img_size)
validation_dataset = image_dataset_from_directory(validation_dir, shuffle=True, batch_size=batch_size, image_size=img_size)

# Define the CNN model
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

# Evaluate on validation data
val_loss, val_acc = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Accuracy per class
class_names = train_dataset.class_names
print(f"Class names: {class_names}")

# Predict and evaluate
class_correct = {name: 0 for name in class_names}
class_total = {name: 0 for name in class_names}

for images, labels in validation_dataset:
    predictions = model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)
    for i in range(len(labels)):
        label = class_names[labels[i].numpy()]
        if predicted_labels[i] == labels[i]:
            class_correct[label] += 1
        class_total[label] += 1

for shape in class_names:
    accuracy = 100 * class_correct[shape] / class_total[shape]
    print(f"Accuracy of {shape}: {accuracy:.2f}%")

print("Validation complete!")

In [ ]:
import os
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")
validation_dir = os.path.join(base_dir, "validation data")

# Create validation directory and class subdirectories if they don't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
for shape in ["circle", "square", "triangle"]:
    shape_dir = os.path.join(validation_dir, shape)
    if not os.path.exists(shape_dir):
        os.makedirs(shape_dir)

# Automatically copy images to validation subdirectories
for shape in ["circle", "square", "triangle"]:
    shape_data_dir = os.path.join(data_dir, shape)
    shape_validation_dir = os.path.join(validation_dir, shape)
    if os.path.exists(shape_data_dir):
        images = [f for f in os.listdir(shape_data_dir) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        selected_images = random.sample(images, min(10, len(images)))
        for image in selected_images:
            shutil.copy(os.path.join(shape_data_dir, image), os.path.join(shape_validation_dir, image))

# Load datasets
batch_size = 32
img_size = (256, 256)

train_dataset = image_dataset_from_directory(data_dir, shuffle=True, batch_size=batch_size, image_size=img_size)
validation_dataset = image_dataset_from_directory(validation_dir, shuffle=True, batch_size=batch_size, image_size=img_size)

# Define the CNN model
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

# Evaluate on validation data
val_loss, val_acc = model.evaluate(validation_dataset)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Accuracy per class
class_names = train_dataset.class_names
print(f"Class names: {class_names}")

# Predict and evaluate
class_correct = {name: 0 for name in class_names}
class_total = {name: 0 for name in class_names}

for images, labels in validation_dataset:
    predictions = model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)
    for i in range(len(labels)):
        label = class_names[labels[i].numpy()]
        if predicted_labels[i] == labels[i]:
            class_correct[label] += 1
        class_total[label] += 1

for shape in class_names:
    accuracy = 100 * class_correct[shape] / class_total[shape]
    print(f"Accuracy of {shape}: {accuracy:.2f}%")

print("Validation complete!")

In [ ]:
import os
import random
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set up directories
base_dir = r"C:/Users/josep"
data_dir = os.path.join(base_dir, "data")
validation_dir = os.path.join(base_dir, "validation data")

# Create validation directory and class subdirectories if they don't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
for shape in ["circle", "square", "triangle"]:
    shape_dir = os.path.join(validation_dir, shape)
    if not os.path.exists(shape_dir):
        os.makedirs(shape_dir)

# Automatically copy images to validation subdirectories
for shape in ["circle", "square", "triangle"]:
    shape_data_dir = os.path.join(data_dir, shape)
    shape_validation_dir = os.path.join(validation_dir, shape)
    if os.path.exists(shape_data_dir):
        images = [f for f in os.listdir(shape_data_dir) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        selected_images = random.sample(images, min(10, len(images)))
        for image in selected_images:
            shutil.copy(os.path.join(shape_data_dir, image), os.path.join(shape_validation_dir, image))

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1))  # Convert grayscale to RGB
])

# Load datasets
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
validation_dataset = datasets.ImageFolder(root=validation_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

# Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(16, 3, kernel_size=3, padding=1),
            nn.Sigmoid(),
            nn.Upsample(scale_factor=2)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the model, loss function, and optimizer
model = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the Autoencoder
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'autoencoder.pth')

# To use the encoder for classification:
class Encoder(nn.Module):
    def __init__(self, autoencoder):
        super(Encoder, self).__init__()
        self.encoder = autoencoder.encoder

    def forward(self, x):
        x = self.encoder(x)
        return x

encoder = Encoder(model).cuda()
# You can now use 'encoder' to extract features and train a classifier on top of these features.